In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import csv
import math
import jieba
import re

In [ ]:
text1 = "It's good, Jinbao says I would like to come again."
text2 = "It's good, Meer says I would like to come again it good."
text3 = "It might be come an excellent one. But I won't come again."
text4 = "It's good, Ginny says I would like to come again."
text5 = "It's good, Penny says I would like to come again."
text6 = "It's good, BadB says I would like to come again."
corpus = tuple([text1, text2, text3, text4, text5, text6])


counter = CountVectorizer(ngram_range=(1,2))
langMod = counter.fit_transform(corpus)

# langMod.toarray()
print(counter.get_feature_names())
query = "It's good"#, Jinbao says I would like to come again."
print('> counter.transform([query]).toarray()')
print(counter.transform([query]).toarray())
print('> langMod.toarray()')
print(langMod.toarray())
print('> Test P( again | come )')
loc0 = counter.vocabulary_.get('come')
loc1 = counter.vocabulary_.get('come again')

print(sum(langMod.toarray().transpose()[loc1]) / sum(langMod.toarray().transpose()[loc0]))

In [11]:
# counter = CountVectorizer(ngram_range=(1,2))
# langMod = counter.fit_transform(corpus)
# langModTrans = langMod.toarray().transpose()

def condProb(bigram, langModTrans):
    cond = bigram.split(' ')[0]
    loc0 = counter.vocabulary_.get(cond)
    loc1 = counter.vocabulary_.get(bigram)
    prob = sum(langModTrans[loc1]) / sum(langModTrans[loc0])
    return prob
# condProb('come again', langModTrans)

def parseSeq(query):
    counterTemp = CountVectorizer(ngram_range=(2,2))
    langModTemp = counterTemp.fit_transform([query])
    bigrams = counterTemp.get_feature_names()
    return bigrams

# text1 = "It's good, Jinbao says I would like to come again."
# parseSeq(text2)

def sequenceProb(query):
    bigrams = parseSeq(query)
#     for bigram in bigrams:
#         print( -math.log(condProb(bigram, langModTrans)))
    logProb = sum(map(lambda x: math.log(condProb(x, langModTrans)), bigrams))
#     logProb += parseSeq(query.split(' ')[0] + ' ' + query.split(' ')[1])[0].split(' ')[0]
    print(math.exp(logProb))
    #     print( sum([math.log(condProb(x, langModTrans)) for x in bigrams]) )

sequenceProb(text1)
sequenceProb(text2)
sequenceProb(text3)
sequenceProb(text4)
sequenceProb(text5)
sequenceProb(text6)

NameError: name 'text1' is not defined

In [ ]:
print(math.log(0.09))
print(math.log(0.9) + math.log(0.1))
print(math.log(0.01))
print(math.log(0.1) + math.log(0.1))

print(sum(map(math.log, [1,3,5])))
a = math.log(0.1) + math.log(0.3) + math.log(0.5)
print(a)
math.exp(a)

In [43]:
data0109 = pd.read_csv('data0109.csv')
data0109 = data0109.dropna()
jieba.load_userdict("/usr/local/lib/python3.5/dist-packages/jieba/user.dict")

# person = '羅婷'
people = list(set(data0109.dropna()['new_Interviewer']))
departs = list(set(data0109.dropna()['部門']))

with open('test.csv', "a") as fp:
    wr = csv.writer(fp, dialect='excel', quotechar = '"')
    wr.writerow(['person', 'date', 'sequenceProb', 'corpusNew'])

for person in people:
    dates = list(set(data0109[data0109['new_Interviewer'] == person]['訪談日期']))
    # records = data0109[data0109['new_Interviewer'] == '賴軒如']['new_GeneralInformation']
    for date in dates:
        records = data0109.loc[(data0109['new_Interviewer'] == person) &
                               (data0109['訪談日期'] == date)]['new_GeneralInformation']

        records = records.dropna()

        corpus = []
        for record in records:
            seg_list = jieba.cut(record, cut_all=False)
            corpus.append(' '.join(seg_list))
        corpusNew = [re.sub(r"[「」（）()]", '', i) for i in corpus]
        corpusNew = [re.sub(r"[一二三四五六七八九] 、", '', i) for i in corpusNew]
        corpusNew = [re.sub(r" [一二三四五六七八九] ", '', i) for i in corpusNew]
        corpusNew = [re.sub(r"[0-9] .", '', i) for i in corpusNew]
        corpusNew = [re.sub(r"[ ]+", ' ', i) for i in corpusNew]
        corpusNew = [re.sub(r"^[ ]", '', i) for i in corpusNew]

        counter = CountVectorizer(ngram_range=(1,2))
        langMod = counter.fit_transform(corpusNew)
        langModTrans = langMod.toarray().transpose()

        for i in range(len(corpusNew)):
            with open('test.csv', "a") as fp:
                wr = csv.writer(fp, dialect='excel', quotechar = '"')
                wr.writerow([person, date, sequenceProb(corpusNew[i]), corpusNew[i]])


In [63]:
test = pd.read_csv('test.csv')
output = pd.merge(test,
                 data0109[['new_Interviewer', '部門']].rename(index=str, columns={"new_Interviewer": "person", "部門": "depart"}),
                 how='left',
                 on='person')
# output.to_csv('output.csv', index=False)
data0109[['new_Interviewer', '部門']].rename(index=str, columns={"new_Interviewer": "person", "部門": "depart"}).shape
# output.shape

(5506, 2)

In [41]:
def condProb(bigram, langModTrans):
    cond = bigram.split(' ')[0]
    loc0 = counter.vocabulary_.get(cond)
    loc1 = counter.vocabulary_.get(bigram)
    prob = sum(langModTrans[loc1]) / sum(langModTrans[loc0])
    return prob
# condProb('come again', langModTrans)

def parseSeq(query):
    counterTemp = CountVectorizer(ngram_range=(2,2))
    langModTemp = counterTemp.fit_transform([query])
    bigrams = counterTemp.get_feature_names()
    return bigrams

# text1 = "It's good, Jinbao says I would like to come again."
# parseSeq(text2)

def sequenceProb(query):
    try:
        bigrams = parseSeq(query)
        logProb = sum(map(lambda x: math.log(condProb(x, langModTrans)), bigrams))
        return math.exp(logProb)
    except:
        return 'Exception'

# counter = CountVectorizer(ngram_range=(1,2))
# langMod = counter.fit_transform(corpusNew)
# langModTrans = langMod.toarray().transpose()

# for i in range(len(corpusNew)):
#     with open('test2.csv', "a") as fp:
#         wr = csv.writer(fp, dialect='excel', quotechar = '"')
#         wr.writerow([sequenceProb(corpusNew[i]), corpusNew[i]])
# parseSeq(corpus[i])

In [40]:
i = 17
print(corpusNew[i])
sequenceProb(corpusNew[i])
# parseSeq(corpus[i])

武樺 國際 是 經貿網 免費會員 主要 出口 至 菲律賓 來電 詢問 經貿網 相關 問題 推廣 經貿網 及國貿 課程
1.143605336741862e-19


In [6]:
# data0109[data0109['new_Interviewer'] == '羅婷']#['new_GeneralInformation']
# list(set(data0109[data0109['new_Interviewer'] == '羅婷']['訪談日期']))
# data0109
data0109 = pd.read_csv('data0109.csv')
list(set(data0109.dropna()['部門']))


['會本部. 產業拓展處',
 '會本部. 台北國際展會營運中心(會)',
 '會本部. 展覽業務處',
 '會本部. 南港國際展覽中心',
 '會本部. 國際企業人才培訓中心',
 '會本部. 市場拓展處',
 '會本部. 市場拓展處(原)',
 '會本部. 服務業推廣中心',
 '會本部. 行銷專案處',
 '會本部. 數位商務處',
 '會本部. 台北國際展會營運中心',
 '會本部. EBS',
 '會本部. 市場研究處(原)',
 '會本部. 資訊及數據中心',
 '會本部. 行政業務處']

In [ ]:
records = records.dropna()
bigText = ''
for record in records:
    bigText += record
bigText

In [ ]:
jieba.analyse.extract_tags(bigText, topK=10, withWeight=True, allowPOS=())

In [18]:
# jieba.analyse.extract_tags(bigText, topK=10, withWeight=True, allowPOS=())
# import jieba.analyse
jieba.load_userdict("/usr/local/lib/python3.5/dist-packages/jieba/user.dict")
# jieba.analyse.TFIDF(idf_path=None)
i = 1
print(records[i])
jieba.analyse.extract_tags(records[i], topK=10, withWeight=True, allowPOS=())


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.851 seconds.
Prefix dict has been built succesfully.


1. 該司產品住要供海事使用。民用產品很少2. 已決定參加2018「高雄國際海事船舶暨國防工業展」，不考慮遊艇展


AttributeError: module 'jieba' has no attribute 'analyse'

In [ ]:
jieba.load_userdict("/usr/local/lib/python3.5/dist-packages/jieba/user.dict")
corpus = []
for record in records:
    seg_list = jieba.cut(record, cut_all=False)
    corpus.append(' '.join(seg_list))
corpus


In [ ]:
import jieba.analyse

jieba.analyse.extract_tags(records[40], topK=3, withWeight=False, allowPOS=())

In [ ]:
counter = CountVectorizer(ngram_range=(1,2))
langMod = counter.fit_transform(corpus)
langModTrans = langMod.toarray().transpose()

def condProb(bigram, langModTrans):
    cond = bigram.split(' ')[0]
    loc0 = counter.vocabulary_.get(cond)
    loc1 = counter.vocabulary_.get(bigram)
    print('{} / {}'.format(sum(langModTrans[loc1]), sum(langModTrans[loc0])))
    prob = sum(langModTrans[loc1]) / sum(langModTrans[loc0])
    return prob
# condProb('come again', langModTrans)

def parseSeq(query):
    counterTemp = CountVectorizer(ngram_range=(2,2))
    langModTemp = counterTemp.fit_transform([query])
    bigrams = counterTemp.get_feature_names()
    return bigrams

# text1 = "It's good, Jinbao says I would like to come again."
# parseSeq(text2)

def sequenceProb(query):
    bigrams = parseSeq(query)
#     for bigram in bigrams:
#         print( -math.log(condProb(bigram, langModTrans)))
    logProb = sum(map(lambda x: math.log(condProb(x, langModTrans)), bigrams))
    print(math.exp(logProb))
    #     print( sum([math.log(condProb(x, langModTrans)) for x in bigrams]) )

sequenceProb(corpus[0])
sequenceProb(corpus[1])
sequenceProb(corpus[2])
sequenceProb(corpus[3])
sequenceProb(corpus[4])

In [ ]:
# condProb('前來 本處', langModTrans)
condProb('巴西 日本', langModTrans)
condProb('海外 機率', langModTrans)


In [ ]:
for i in range(1,10):
    print('')
    for j in range(1,10):
        print('%2d' % (i * j), end=' ')

for i in range(100):
    if int(i % 10) == 0: print('')
    if int(i / 10) * (i % 10) != 0:
        print('%2d' % (int(i / 10) * (i % 10)), end=' ')

In [ ]:
# 1 , 1 , 2 , 3 , 5 , 8 , 13 , 21 , 34 , 55 , 89 , 144
def main(n):
    def feb(n):
        if n == 0:
            return 1
        if n == 1:
            return 1
        if febList[n]:
            return febList[n]
        ans = feb(n - 1) + feb(n - 2)
        febList[n] = ans
        return ans
    febList = [0] * (n+1)
    answer = feb(n)
    print(febList)
    return answer
# main(144)

def feb(n):
    febList = [1] * (n)
    for i in range(2,n):
        febList[i] = febList[i-1] + febList[i-2]
    return febList[n-1]
feb(288)

In [ ]:
comp104 = pd.read_csv('company104_20180307.csv')
records = comp104['description']

jieba.load_userdict("/usr/local/lib/python3.5/dist-packages/jieba/comp104.dict")
comp104['corpus'] = pd.Series('')

for i in range(len(records)):
    try:
        seg_list = jieba.cut(records[i], cut_all=False)
        comp104.at[i,'corpus'] = ' '.join(seg_list)
    except:
        continue
comp104

In [ ]:
comp104 = comp104.dropna()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(comp104['corpus'].dropna())

In [ ]:
x = tfs.toarray()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=8).fit(tfs)

In [ ]:
comp104['label'] = pd.Series(kmeans.labels_)


In [ ]:
comp104[comp104['label']==5.0]

In [ ]:
kmeans.cluster_centers_

In [ ]:
# clustering dataset
# determine k using elbow method

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

# create new plot and data
plt.plot()
X = tfs
colors = ['b', 'g', 'r']
markers = ['o', 'v', 's']

# k means determine k
distortions = []
K = range(40,60)
for k in K:
    print(k)
    kmeanModel = KMeans(n_clusters=k).fit(X)
    distortions.append(sum(np.min(pairwise_distances(X, kmeanModel.cluster_centers_, metric='euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1))
# X.shape[0]
from sklearn.metrics.pairwise import pairwise_distances
pairwise_distances(X, kmeanModel.cluster_centers_, metric='euclidean')